<a href="https://colab.research.google.com/github/gameaphiwat/data_viz_2022/blob/main/Final_Exam_DataViz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Exam: งานกลุ่ม Final ที่อาจารย์สุ่มให้
ให้นักศึกษาสร้าง Dashboard แสดงรายงานเกี่ยวกับข้อมูลการตรวจโรค(ov) จาก ไฟล์ alldb.csv และ hospital_list.xlsx

( ดาวน์โหลดจาก https://drive.google.com/.../1D... )
* โดยข้อมูลหลักจะอยู่ใน ตาราง alldb.csv (ข้อมูลที่สมบูรณ์ที่สามารถนำมาใช้งานได้ต้องมีค่าใน column ต่อไปนี้ดังนี้
* user เป็นตัวเลข 5 หลัก (ตัวเลขเท่านั้น)
* RowKey เป็น String
* date เป็น YYYY-MM-DD
* phone เป็น String
* site เป็นตัวเลข 5 หลัก (ตัวเลขเท่านั้น)
* time เป็น hh-mm-ss
* urine มีค่า String
* ov เป็นตัวเลข มีค่าตั้งแต่ 0-4 (0 คือไม่พบโรค 1-4 คือพบโรคที่ระดับความรุนแรงต่างๆ)
* PartitionKey เป็นตัวเลข 5 หลัก (ตัวเลขเท่านั้น))

ไฟล์ alldb.csv และ hospital_list.xlsx สามารถถูกเชื่อมโยงกันโดย site ในไฟล์ alldb.csv คือ hcode ในไฟล์ hospital_list.xlsx
โดยข้อมูลสำคัญที่ต้องแสดงคือ
* จำนวนการตรวจในแต่ละพื้นที่ (จังหวัด-อำเภอ ในไฟล์ hospital_list.xlsx) เทียบกับ จำนวนที่พบโรค (column ov ในไฟล์ alldb.csv)
* ช่วงเวลาที่ตรวจ (date ในไฟล์ alldb.csv)
* อื่นๆที่นักศึกษาเห็นว่าเป็นประโยชน์ (phone (ชนิดโทรศัพท์ที่ใช้บันทึก), title (คำนำหน้าชื่อ), age (อายุ))

(คิดคะแนนตามกลุ่ม final โดยไฟล์ที่ทุกคนในกลุ่มต้องมีคือไฟล์ data pre-processing และ link ไปที่ Dashboard ที่สร้่างโดยเปิดให้สามารถเข้าไปดูและใช้งานได้)

**ส่งไม่เกิน 23.59 วันจันทร์ที่ 11 เมษายน 2565**

#**Load data**

In [6]:
from google.colab import drive ## เชื่อมต่อ Drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os ## import os เลือกเส้นทางในการเชื่อมต่อ ทำการเลือก folder ที่ใช้เก็บ data ในที่นี้คือ Final_Exam
path = '/content/drive/My Drive/DataViz22_data'

In [8]:
import pandas as pd ## ทำการ import pandas ในชื่อ pd เพื่อใช้ในการเขียนโค้ด
import numpy as np ##ทำการ import numpy ในชื่อ np เพื่อใช้ในการเขียนโค้ด

## การตรวจโรค(ov)

In [9]:
alldb =  pd.read_csv(os.path.join(path,'alldb.csv')) ## ทำการนำเข้าข้อมูล การตรวจโรค(ov) โดยการเรียกใช้ข้อมูลประเภท .csv ใน drive ไฟล์ที่เรียกใช้คือ alldb.csv # โดยตั้งชื่ตารางว่า alldb
alldb.head() ## แสดงตาราง alldb

,Unnamed: 0,user,RowKey,care,date,ov,phone,site,time,urine,pid,PartitionKey,title,age
0,0,test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,data is not completed,data is not completed
1,1,04857,136FD222-FD9F-4E54-AE1F-7282A276B085,NaN,2022-02-24,NaN,"iPhone10,4",05310,10:50:32,NaN,NaN,NaN,data is not completed,data is not completed
2,2,NaN,1E6D3D62-4C8A-4F8E-90D1-0DC5BC2752DF.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,data is not completed,data is not completed
3,3,3450101576642,2D331FB0-CBA2-4586-A5B0-CB3CF44DD9FE,NaN,2022-02-23,0,"iPhone13,4",05087,09:59:47,ปกติ (เหลืองใส),NaN,NaN,data is not completed,data is not completed
4,4,3450101576642,45197AF7-7CB4-470C-9ABB-37C092BD6B21,NaN,2022-02-23,NaN,"iPhone13,4",05087,09:44:06,ปกติ (เหลืองใส),NaN,NaN,data is not completed,data is not completed


In [10]:
alldb.shape#ดูขนาด

(40035, 14)

In [11]:
alldb = alldb.drop(columns=['Unnamed: 0','care','pid'])     # ลบ คอลัมน์ที่ไม่ต้องการออก
print(alldb.shape)
alldb.head()

(40035, 11)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,title,age
0,test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,data is not completed,data is not completed
1,04857,136FD222-FD9F-4E54-AE1F-7282A276B085,2022-02-24,NaN,"iPhone10,4",05310,10:50:32,NaN,NaN,data is not completed,data is not completed
2,NaN,1E6D3D62-4C8A-4F8E-90D1-0DC5BC2752DF.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,data is not completed,data is not completed
3,3450101576642,2D331FB0-CBA2-4586-A5B0-CB3CF44DD9FE,2022-02-23,0,"iPhone13,4",05087,09:59:47,ปกติ (เหลืองใส),NaN,data is not completed,data is not completed
4,3450101576642,45197AF7-7CB4-470C-9ABB-37C092BD6B21,2022-02-23,NaN,"iPhone13,4",05087,09:44:06,ปกติ (เหลืองใส),NaN,data is not completed,data is not completed


In [12]:
alldb.isnull().sum()

user            762
RowKey            1
date            554
ov              364
phone             3
site             99
time            557
urine           253
PartitionKey    185
title           444
age             542
dtype: int64

In [13]:
alldb.dtypes  

user            object
RowKey          object
date            object
ov              object
phone           object
site            object
time            object
urine           object
PartitionKey    object
title           object
age             object
dtype: object

### เปลี่ยนข้อมูลให้เป็นไปตามข้อกำหนด

user เป็นตัวเลข 5 หลัก (ตัวเลขเท่านั้น)

In [14]:
alldb = alldb[(alldb['user'].astype(str).str.isdigit()) & (alldb['user'].str.len() == 5)]
print(alldb.shape)
alldb.head()

(38508, 11)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,title,age
1,04857,136FD222-FD9F-4E54-AE1F-7282A276B085,2022-02-24,NaN,"iPhone10,4",05310,10:50:32,NaN,NaN,data is not completed,data is not completed
5,08777,472EED3E-E2A7-4551-9249-D87747C68AFD,2022-02-25,NaN,"iPhone13,2",05248,10:14:46,NaN,NaN,data is not completed,data is not completed
7,04857,73E96EE6-7660-4204-9914-92112603C514,2022-02-24,NaN,"iPhone10,4",05310,11:14:21,NaN,NaN,data is not completed,data is not completed
11,08777,C23A546B-EB6E-4DB4-93FA-2F9A7A85AEE9,2022-02-25,NaN,"iPhone13,2",05248,10:14:32,NaN,NaN,data is not completed,data is not completed
12,11111,C42C6F97-4388-4C03-A0AB-84F21FAED2C2,2022-02-27,NaN,"iPhone13,3",22222,17:39:35,NaN,NaN,data is not completed,data is not completed


ที่มา: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.isdigit.html?fbclid=IwAR3kGQxrnmeiiGTt2yBByR_BhjwT3fRJyHLfiHBzD1d3IkZAjWLlmXYitvQ

ที่มา: https://stackoverflow.com/questions/24883503/pandas-select-only-numeric-or-integer-field-from-dataframe?fbclid=IwAR1rena-MZiy2c_gGymqXvb2e2XhZEArvw7JgKGjHAB3e2wWT5cPa2T0zEI

In [15]:
alldb.isnull().sum()  ## ดูค่าว่าง

user              0
RowKey            0
date            302
ov              221
phone             0
site              2
time            303
urine           126
PartitionKey     25
title           413
age             511
dtype: int64

###**RowKey เป็น String**

In [16]:
alldb['RowKey'] = alldb['RowKey'].astype('str')
print(alldb.shape)
alldb.head()

(38508, 11)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,title,age
1,04857,136FD222-FD9F-4E54-AE1F-7282A276B085,2022-02-24,NaN,"iPhone10,4",05310,10:50:32,NaN,NaN,data is not completed,data is not completed
5,08777,472EED3E-E2A7-4551-9249-D87747C68AFD,2022-02-25,NaN,"iPhone13,2",05248,10:14:46,NaN,NaN,data is not completed,data is not completed
7,04857,73E96EE6-7660-4204-9914-92112603C514,2022-02-24,NaN,"iPhone10,4",05310,11:14:21,NaN,NaN,data is not completed,data is not completed
11,08777,C23A546B-EB6E-4DB4-93FA-2F9A7A85AEE9,2022-02-25,NaN,"iPhone13,2",05248,10:14:32,NaN,NaN,data is not completed,data is not completed
12,11111,C42C6F97-4388-4C03-A0AB-84F21FAED2C2,2022-02-27,NaN,"iPhone13,3",22222,17:39:35,NaN,NaN,data is not completed,data is not completed


###**date เป็น YYYY-MM-DD**

In [17]:
import datetime ## ทำการ import package datetime 
alldb = pd.DataFrame(alldb) 
alldb['date'] = pd.to_datetime(alldb['date'], format = "%Y-%m-%d") 
print(alldb.shape)
alldb.head()

(38508, 11)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,title,age
1,04857,136FD222-FD9F-4E54-AE1F-7282A276B085,2022-02-24,NaN,"iPhone10,4",05310,10:50:32,NaN,NaN,data is not completed,data is not completed
5,08777,472EED3E-E2A7-4551-9249-D87747C68AFD,2022-02-25,NaN,"iPhone13,2",05248,10:14:46,NaN,NaN,data is not completed,data is not completed
7,04857,73E96EE6-7660-4204-9914-92112603C514,2022-02-24,NaN,"iPhone10,4",05310,11:14:21,NaN,NaN,data is not completed,data is not completed
11,08777,C23A546B-EB6E-4DB4-93FA-2F9A7A85AEE9,2022-02-25,NaN,"iPhone13,2",05248,10:14:32,NaN,NaN,data is not completed,data is not completed
12,11111,C42C6F97-4388-4C03-A0AB-84F21FAED2C2,2022-02-27,NaN,"iPhone13,3",22222,17:39:35,NaN,NaN,data is not completed,data is not completed


In [18]:
alldb['date'] = alldb[['date']].fillna(method="ffill")      # เติมค่าว่างของวันที่ด้วย ค่าก่อนหน้า หรือค่าข้างบน
print(alldb.shape)
alldb.head()

(38508, 11)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,title,age
1,04857,136FD222-FD9F-4E54-AE1F-7282A276B085,2022-02-24,NaN,"iPhone10,4",05310,10:50:32,NaN,NaN,data is not completed,data is not completed
5,08777,472EED3E-E2A7-4551-9249-D87747C68AFD,2022-02-25,NaN,"iPhone13,2",05248,10:14:46,NaN,NaN,data is not completed,data is not completed
7,04857,73E96EE6-7660-4204-9914-92112603C514,2022-02-24,NaN,"iPhone10,4",05310,11:14:21,NaN,NaN,data is not completed,data is not completed
11,08777,C23A546B-EB6E-4DB4-93FA-2F9A7A85AEE9,2022-02-25,NaN,"iPhone13,2",05248,10:14:32,NaN,NaN,data is not completed,data is not completed
12,11111,C42C6F97-4388-4C03-A0AB-84F21FAED2C2,2022-02-27,NaN,"iPhone13,3",22222,17:39:35,NaN,NaN,data is not completed,data is not completed


###**phone เป็น String**

In [19]:
alldb['phone'] = alldb['phone'].astype('str')
print(alldb.shape)
alldb.head()

(38508, 11)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,title,age
1,04857,136FD222-FD9F-4E54-AE1F-7282A276B085,2022-02-24,NaN,"iPhone10,4",05310,10:50:32,NaN,NaN,data is not completed,data is not completed
5,08777,472EED3E-E2A7-4551-9249-D87747C68AFD,2022-02-25,NaN,"iPhone13,2",05248,10:14:46,NaN,NaN,data is not completed,data is not completed
7,04857,73E96EE6-7660-4204-9914-92112603C514,2022-02-24,NaN,"iPhone10,4",05310,11:14:21,NaN,NaN,data is not completed,data is not completed
11,08777,C23A546B-EB6E-4DB4-93FA-2F9A7A85AEE9,2022-02-25,NaN,"iPhone13,2",05248,10:14:32,NaN,NaN,data is not completed,data is not completed
12,11111,C42C6F97-4388-4C03-A0AB-84F21FAED2C2,2022-02-27,NaN,"iPhone13,3",22222,17:39:35,NaN,NaN,data is not completed,data is not completed


###**site เป็นตัวเลข 5 หลัก (ตัวเลขเท่านั้น)**

In [20]:
alldb = alldb[(alldb['site'].astype(str).str.isdigit()) & (alldb['site'].str.len() == 5)]  ## .str.isdigit ใช้ตรวจสอบว่าอักขระทั้งหมดเป็นตัวเลขหรือไม่
print(alldb.shape) 
alldb.head()

(38493, 11)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,title,age
1,04857,136FD222-FD9F-4E54-AE1F-7282A276B085,2022-02-24,NaN,"iPhone10,4",05310,10:50:32,NaN,NaN,data is not completed,data is not completed
5,08777,472EED3E-E2A7-4551-9249-D87747C68AFD,2022-02-25,NaN,"iPhone13,2",05248,10:14:46,NaN,NaN,data is not completed,data is not completed
7,04857,73E96EE6-7660-4204-9914-92112603C514,2022-02-24,NaN,"iPhone10,4",05310,11:14:21,NaN,NaN,data is not completed,data is not completed
11,08777,C23A546B-EB6E-4DB4-93FA-2F9A7A85AEE9,2022-02-25,NaN,"iPhone13,2",05248,10:14:32,NaN,NaN,data is not completed,data is not completed
12,11111,C42C6F97-4388-4C03-A0AB-84F21FAED2C2,2022-02-27,NaN,"iPhone13,3",22222,17:39:35,NaN,NaN,data is not completed,data is not completed


In [21]:
alldb.isnull().sum()      ## ดูค่าว่าง

user              0
RowKey            0
date              0
ov              219
phone             0
site              0
time            301
urine           124
PartitionKey     20
title           409
age             507
dtype: int64

###**time เป็น hh-mm-ss**

In [22]:
alldb['time'] = alldb[['time']].fillna(method="ffill")      # เติมค่าว่างของเวลาด้วย ค่าก่อนหน้า หรือค่าข้างบน
print(alldb.shape)
alldb.head()

(38493, 11)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,title,age
1,04857,136FD222-FD9F-4E54-AE1F-7282A276B085,2022-02-24,NaN,"iPhone10,4",05310,10:50:32,NaN,NaN,data is not completed,data is not completed
5,08777,472EED3E-E2A7-4551-9249-D87747C68AFD,2022-02-25,NaN,"iPhone13,2",05248,10:14:46,NaN,NaN,data is not completed,data is not completed
7,04857,73E96EE6-7660-4204-9914-92112603C514,2022-02-24,NaN,"iPhone10,4",05310,11:14:21,NaN,NaN,data is not completed,data is not completed
11,08777,C23A546B-EB6E-4DB4-93FA-2F9A7A85AEE9,2022-02-25,NaN,"iPhone13,2",05248,10:14:32,NaN,NaN,data is not completed,data is not completed
12,11111,C42C6F97-4388-4C03-A0AB-84F21FAED2C2,2022-02-27,NaN,"iPhone13,3",22222,17:39:35,NaN,NaN,data is not completed,data is not completed


In [23]:
import datetime ## ทำการ import package datetime 
alldb = pd.DataFrame(alldb) 
alldb['time'] = pd.to_datetime(alldb['time'],format = '%H:%M:%S').dt.time
print(alldb.shape)
alldb.head()

(38493, 11)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,title,age
1,04857,136FD222-FD9F-4E54-AE1F-7282A276B085,2022-02-24,NaN,"iPhone10,4",05310,10:50:32,NaN,NaN,data is not completed,data is not completed
5,08777,472EED3E-E2A7-4551-9249-D87747C68AFD,2022-02-25,NaN,"iPhone13,2",05248,10:14:46,NaN,NaN,data is not completed,data is not completed
7,04857,73E96EE6-7660-4204-9914-92112603C514,2022-02-24,NaN,"iPhone10,4",05310,11:14:21,NaN,NaN,data is not completed,data is not completed
11,08777,C23A546B-EB6E-4DB4-93FA-2F9A7A85AEE9,2022-02-25,NaN,"iPhone13,2",05248,10:14:32,NaN,NaN,data is not completed,data is not completed
12,11111,C42C6F97-4388-4C03-A0AB-84F21FAED2C2,2022-02-27,NaN,"iPhone13,3",22222,17:39:35,NaN,NaN,data is not completed,data is not completed


###**urine มีค่า String**

In [24]:
alldb['urine'] = alldb[['urine']].fillna(method="ffill")      # เติมค่าว่างของ urine(ปัสสาวะ) ด้วย ค่าก่อนหน้า หรือค่าข้างบน
print(alldb.shape)
alldb.head()

(38493, 11)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,title,age
1,04857,136FD222-FD9F-4E54-AE1F-7282A276B085,2022-02-24,NaN,"iPhone10,4",05310,10:50:32,NaN,NaN,data is not completed,data is not completed
5,08777,472EED3E-E2A7-4551-9249-D87747C68AFD,2022-02-25,NaN,"iPhone13,2",05248,10:14:46,NaN,NaN,data is not completed,data is not completed
7,04857,73E96EE6-7660-4204-9914-92112603C514,2022-02-24,NaN,"iPhone10,4",05310,11:14:21,NaN,NaN,data is not completed,data is not completed
11,08777,C23A546B-EB6E-4DB4-93FA-2F9A7A85AEE9,2022-02-25,NaN,"iPhone13,2",05248,10:14:32,NaN,NaN,data is not completed,data is not completed
12,11111,C42C6F97-4388-4C03-A0AB-84F21FAED2C2,2022-02-27,NaN,"iPhone13,3",22222,17:39:35,NaN,NaN,data is not completed,data is not completed


In [25]:
alldb['urine'] = alldb['urine'].astype('str')
print(alldb.shape)
alldb.head()

(38493, 11)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,title,age
1,04857,136FD222-FD9F-4E54-AE1F-7282A276B085,2022-02-24,NaN,"iPhone10,4",05310,10:50:32,nan,NaN,data is not completed,data is not completed
5,08777,472EED3E-E2A7-4551-9249-D87747C68AFD,2022-02-25,NaN,"iPhone13,2",05248,10:14:46,nan,NaN,data is not completed,data is not completed
7,04857,73E96EE6-7660-4204-9914-92112603C514,2022-02-24,NaN,"iPhone10,4",05310,11:14:21,nan,NaN,data is not completed,data is not completed
11,08777,C23A546B-EB6E-4DB4-93FA-2F9A7A85AEE9,2022-02-25,NaN,"iPhone13,2",05248,10:14:32,nan,NaN,data is not completed,data is not completed
12,11111,C42C6F97-4388-4C03-A0AB-84F21FAED2C2,2022-02-27,NaN,"iPhone13,3",22222,17:39:35,nan,NaN,data is not completed,data is not completed


###**ov เป็นตัวเลข มีค่าตั้งแต่ 0-4 (0 คือไม่พบโรค 1-4 คือพบโรคที่ระดับความรุนแรงต่างๆ)**

In [26]:
alldb = alldb.dropna(subset=['ov'])
print(alldb.shape)
alldb.head()

(38274, 11)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,title,age
38,48368,cttsajptww.png,2022-02-24,1,OPPO CPH1969,05081,10:31:10,ปกติ (เหลืองใส),NaN,data is not completed,data is not completed
44,40881,dzvoblimkf.png,2022-02-26,2,HUAWEI STK-L22,05435,09:32:32,ปกติ (เหลืองใส),NaN,data is not completed,data is not completed
54,56845,ftuhzneacx.png,2022-03-02,0,Samsung SM-A505F,05440,10:09:13,สีเหลืองส้ม (ผิดปกติ),NaN,data is not completed,data is not completed
72,99250,hynbuxzgpf.png,2022-02-27,3,Xiaomi 2109119DG,11066,16:12:11,ปกติ (เหลืองใส),NaN,data is not completed,data is not completed
74,77645,iciurjxsbw.png,2022-02-25,0,Samsung SM-N960F,05084,13:14:20,ปกติ (เหลืองใส),NaN,data is not completed,data is not completed


In [27]:
set(alldb['ov'])

{'0', '1', '2', '3', '4', 'ติดเชื้อ +2 '}

In [28]:
alldb['ov'].replace({'ติดเชื้อ +2 ':'1'}, inplace=True)

In [29]:
set(alldb['ov'])

{'0', '1', '2', '3', '4'}

In [30]:
alldb['ov'] = alldb['ov'].astype('int')
print(alldb.shape)
alldb.head()

(38274, 11)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,title,age
38,48368,cttsajptww.png,2022-02-24,1,OPPO CPH1969,05081,10:31:10,ปกติ (เหลืองใส),NaN,data is not completed,data is not completed
44,40881,dzvoblimkf.png,2022-02-26,2,HUAWEI STK-L22,05435,09:32:32,ปกติ (เหลืองใส),NaN,data is not completed,data is not completed
54,56845,ftuhzneacx.png,2022-03-02,0,Samsung SM-A505F,05440,10:09:13,สีเหลืองส้ม (ผิดปกติ),NaN,data is not completed,data is not completed
72,99250,hynbuxzgpf.png,2022-02-27,3,Xiaomi 2109119DG,11066,16:12:11,ปกติ (เหลืองใส),NaN,data is not completed,data is not completed
74,77645,iciurjxsbw.png,2022-02-25,0,Samsung SM-N960F,05084,13:14:20,ปกติ (เหลืองใส),NaN,data is not completed,data is not completed


In [31]:
alldb.isnull().sum() 

user              0
RowKey            0
date              0
ov                0
phone             0
site              0
time              0
urine             0
PartitionKey     14
title           408
age             505
dtype: int64

###**PartitionKey เป็นตัวเลข 5 หลัก (ตัวเลขเท่านั้น)**

In [32]:
alldb = alldb[(alldb['PartitionKey'].astype(str).str.isdigit()) & (alldb['PartitionKey'].str.len() == 5)]
print(alldb.shape)
alldb.head()

(38172, 11)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,title,age
190,06223,kfpwuaqldj.png,2022-03-23,0,Xiaomi 21081111RG,13777,16:31:45,ปกติ (เหลืองใส),00000,NaN,NaN
191,12015,00859DB9-1BD9-4825-B246-3DC25EFD1924,2022-03-30,4,"iPhone12,3",06307,14:46:08,ปกติ (เหลืองใส),00001,นางสาว,47
192,91811,1376ACA2-AFA5-4457-967A-D2FCF908E6A5,2022-03-30,4,"iPhone14,3",05328,15:27:45,ปกติ (เหลืองใส),00001,นาง,41
193,95821,1914EE67-CD8F-47EE-A968-2772911AA5DF,2022-03-30,3,"iPhone13,4",06302,10:16:46,ปกติ (เหลืองใส),00001,นาง,55
194,13361,1A219EE7-3647-4D7D-A8C0-AE638E69D889,2022-03-29,3,"iPhone9,4",05059,12:26:38,ปกติ (เหลืองใส),00001,นาย,66


###**title (sex) เปลี่ยนข้อมูลเป็น ชาย และ หญิง**


In [33]:
alldb = alldb.rename(columns={'title':'sex'})  #     เปลี่ยนชื่อ คอลัมน์ title เป็น sex
print(alldb.shape)
alldb.head()

(38172, 11)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,sex,age
190,06223,kfpwuaqldj.png,2022-03-23,0,Xiaomi 21081111RG,13777,16:31:45,ปกติ (เหลืองใส),00000,NaN,NaN
191,12015,00859DB9-1BD9-4825-B246-3DC25EFD1924,2022-03-30,4,"iPhone12,3",06307,14:46:08,ปกติ (เหลืองใส),00001,นางสาว,47
192,91811,1376ACA2-AFA5-4457-967A-D2FCF908E6A5,2022-03-30,4,"iPhone14,3",05328,15:27:45,ปกติ (เหลืองใส),00001,นาง,41
193,95821,1914EE67-CD8F-47EE-A968-2772911AA5DF,2022-03-30,3,"iPhone13,4",06302,10:16:46,ปกติ (เหลืองใส),00001,นาง,55
194,13361,1A219EE7-3647-4D7D-A8C0-AE638E69D889,2022-03-29,3,"iPhone9,4",05059,12:26:38,ปกติ (เหลืองใส),00001,นาย,66


In [34]:
set(alldb['sex'])   # ดู set ของข้อมูล

{'  นาง',
 ' นาง',
 ' นางสาว',
 ' นาย',
 'Mrs',
 nan,
 'ชาย',
 'ด.ช.',
 'ด.ญ.',
 'ด.ต.',
 'น',
 'น.ส',
 'น.ส ',
 'น.ส.',
 'นส',
 'นส.',
 'นส. ',
 'นาง',
 'นาง ',
 'นาง.',
 'นางl',
 'นางสาว',
 'นางสาว ',
 'นางสาว                   ',
 'นาน',
 'นาบ',
 'นาย',
 'นาย ',
 'นาย  ',
 'นาว',
 'นาวสาว',
 'นา่ง',
 'น่าย',
 'พจอ.',
 'พระ',
 'พระครุ',
 'พระปลัด',
 'พลฯ',
 'ร.ต.',
 'ร.ต.ต.',
 'ร.ต.ท.',
 'ร.ต.อ.',
 'รต.',
 'ว่าที่ ร.ต.',
 'ว่าที่ ร.ท.',
 'ว่าที่ร้อยตรี',
 'ส.อ.',
 'ืนาง',
 'เด็กชาย',
 'เด็กหญิง'}

In [35]:
alldb['sex'].replace({'  นาง':'หญิง',
 ' นาง':'หญิง',
 ' นางสาว':'หญิง',
 ' นาย':'ชาย',
 'Mrs':'หญิง',
 'ชาย':'ชาย',
 'ด.ช.':'ชาย',
 'ด.ญ.':'หญิง',
 'ด.ต.':'ไม่ระบุ',
 'น':'ไม่ระบุ',
 'น.ส':'หญิง',
 'น.ส ':'หญิง',
 'น.ส.':'หญิง',
 'นส':'หญิง',
 'นส.':'หญิง',
 'นส. ':'หญิง',
 'นาง':'หญิง',
 'นาง ':'หญิง',
 'นาง.':'หญิง',
 'นางl':'หญิง',
 'นางสาว':'หญิง',
 'นางสาว ':'หญิง',
 'นางสาว                   ':'หญิง',
 'นาน':'ชาย',
 'นาบ':'ชาย',
 'นาย':'ชาย',
 'นาย ':'ชาย',
 'นาย  ':'ชาย',
 'นาว':'หญิง',
 'นาวสาว':'หญิง',
 'นา่ง':'หญิง',
 'น่าย':'ชาย',
 'พจอ.':'ไม่ระบุ',
 'พระ':'ไม่ระบุ',
 'พระครุ':'ไม่ระบุ',
 'พระปลัด':'ไม่ระบุ',
 'พลฯ':'ไม่ระบุ',
 'ร.ต.':'ไม่ระบุ',
 'ร.ต.ต.':'ไม่ระบุ',
 'ร.ต.ท.':'ไม่ระบุ',
 'ร.ต.อ.':'ไม่ระบุ',
 'รต.':'ไม่ระบุ',
 'ว่าที่ ร.ต.':'ไม่ระบุ',
 'ว่าที่ ร.ท.':'ไม่ระบุ',
 'ว่าที่ร้อยตรี':'ไม่ระบุ',
 'ส.อ.':'ไม่ระบุ',
 'ืนาง':'หญิง',
 'เด็กชาย':'ชาย',
 'เด็กหญิง':'หญิง'}, inplace=True)

In [36]:
set(alldb['sex'])

{nan, 'ชาย', 'หญิง', 'ไม่ระบุ'}

In [37]:
alldb['sex'] = alldb[['sex']].fillna('ไม่ระบุ')      # เติมค่าว่างของ sex ด้วย ไม่ระบุ
print(alldb.shape)
alldb.head()

(38172, 11)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,sex,age
190,06223,kfpwuaqldj.png,2022-03-23,0,Xiaomi 21081111RG,13777,16:31:45,ปกติ (เหลืองใส),00000,ไม่ระบุ,NaN
191,12015,00859DB9-1BD9-4825-B246-3DC25EFD1924,2022-03-30,4,"iPhone12,3",06307,14:46:08,ปกติ (เหลืองใส),00001,หญิง,47
192,91811,1376ACA2-AFA5-4457-967A-D2FCF908E6A5,2022-03-30,4,"iPhone14,3",05328,15:27:45,ปกติ (เหลืองใส),00001,หญิง,41
193,95821,1914EE67-CD8F-47EE-A968-2772911AA5DF,2022-03-30,3,"iPhone13,4",06302,10:16:46,ปกติ (เหลืองใส),00001,หญิง,55
194,13361,1A219EE7-3647-4D7D-A8C0-AE638E69D889,2022-03-29,3,"iPhone9,4",05059,12:26:38,ปกติ (เหลืองใส),00001,ชาย,66


In [38]:
alldb.isnull().sum()      # ดูค่าว่าง

user              0
RowKey            0
date              0
ov                0
phone             0
site              0
time              0
urine             0
PartitionKey      0
sex               0
age             503
dtype: int64

###**age**

In [39]:
alldb['age'] = alldb['age'].astype(float).astype('Int64') 

In [41]:
mean_age = alldb['age'].mean()      # หาอายุเฉลี่ย
mean_age

53.95614586607985

In [40]:
alldb['age'] = alldb['age'].fillna(54)    

In [42]:
alldb.isnull().sum()      # ดูค่าว่าง

user            0
RowKey          0
date            0
ov              0
phone           0
site            0
time            0
urine           0
PartitionKey    0
sex             0
age             0
dtype: int64

In [43]:
alldb.dtypes  

user                    object
RowKey                  object
date            datetime64[ns]
ov                       int64
phone                   object
site                    object
time                    object
urine                   object
PartitionKey            object
sex                     object
age                      Int64
dtype: object

###**ตารางที่ clean เสร็จแล้ว**

In [44]:
print(alldb.shape)      
alldb.head()

(38172, 11)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,sex,age
190,06223,kfpwuaqldj.png,2022-03-23,0,Xiaomi 21081111RG,13777,16:31:45,ปกติ (เหลืองใส),00000,ไม่ระบุ,54
191,12015,00859DB9-1BD9-4825-B246-3DC25EFD1924,2022-03-30,4,"iPhone12,3",06307,14:46:08,ปกติ (เหลืองใส),00001,หญิง,47
192,91811,1376ACA2-AFA5-4457-967A-D2FCF908E6A5,2022-03-30,4,"iPhone14,3",05328,15:27:45,ปกติ (เหลืองใส),00001,หญิง,41
193,95821,1914EE67-CD8F-47EE-A968-2772911AA5DF,2022-03-30,3,"iPhone13,4",06302,10:16:46,ปกติ (เหลืองใส),00001,หญิง,55
194,13361,1A219EE7-3647-4D7D-A8C0-AE638E69D889,2022-03-29,3,"iPhone9,4",05059,12:26:38,ปกติ (เหลืองใส),00001,ชาย,66


#**ข้อมูลโรงพยาบาล** 

In [45]:
hospital_list =  pd.read_excel(os.path.join(path,'hospital_list.xlsx')) ## ทำการนำเข้าข้อมูล โรงพยาบาล โดยการเรียกใช้ข้อมูลประเภท .xlsx ใน drive ไฟล์ที่เรียกใช้คือ hospital_list.xlsx # โดยตั้งชื่ตารางว่า hospital_list
print(hospital_list.shape)   
hospital_list.head() ## แสดงตาราง hospital_list

(17103, 16)


,hcode,name,zone_code,zone_name,provincecode,province,amphurcode,amphur,tamboncode,tambon,code4,name5,code9,name10,short_thai,short_english
0,00001,สำนักงานสาธารณสุขจังหวัดสมุทรปราการ,6,สาขาพื้นที่(ระยอง),11.0,สมุทรปราการ,1.0,เมืองสมุทรปราการ,1.0,ปากน้ำ,1.0,สำนักงานสาธารณสุขจังหวัด,NaN,NaN,สป,SPK
1,00002,สำนักงานสาธารณสุขจังหวัดนนทบุรี,4,สาขาพื้นที่(สระบุรี),12.0,นนทบุรี,1.0,เมืองนนทบุรี,4.0,บางกระสอ,1.0,สำนักงานสาธารณสุขจังหวัด,NaN,NaN,นบ,NBI
2,00003,สำนักงานสาธารณสุขจังหวัดปทุมธานี,4,สาขาพื้นที่(สระบุรี),13.0,ปทุมธานี,1.0,เมืองปทุมธานี,1.0,บางปรอก,1.0,สำนักงานสาธารณสุขจังหวัด,NaN,NaN,ปท,PTE
3,00004,สำนักงานสาธารณสุขจังหวัดพระนครศรีอยุธยา,4,สาขาพื้นที่(สระบุรี),14.0,พระนครศรีอยุธยา,1.0,พระนครศรีอยุธยา,3.0,หอรัตนไชย,1.0,สำนักงานสาธารณสุขจังหวัด,NaN,NaN,อย,AYA
4,00005,สำนักงานสาธารณสุขจังหวัดอ่างทอง,4,สาขาพื้นที่(สระบุรี),15.0,อ่างทอง,1.0,เมืองอ่างทอง,8.0,บ้านอิฐ,1.0,สำนักงานสาธารณสุขจังหวัด,NaN,NaN,อท,ATG


In [46]:
hospital = hospital_list[['hcode','name','zone_name','province','amphur','tambon','name5']]      # เลือกเฉพาะคอลัมน์ที่ต้องการ
print(hospital.shape)
hospital.head()

(17103, 7)


,hcode,name,zone_name,province,amphur,tambon,name5
0,00001,สำนักงานสาธารณสุขจังหวัดสมุทรปราการ,สาขาพื้นที่(ระยอง),สมุทรปราการ,เมืองสมุทรปราการ,ปากน้ำ,สำนักงานสาธารณสุขจังหวัด
1,00002,สำนักงานสาธารณสุขจังหวัดนนทบุรี,สาขาพื้นที่(สระบุรี),นนทบุรี,เมืองนนทบุรี,บางกระสอ,สำนักงานสาธารณสุขจังหวัด
2,00003,สำนักงานสาธารณสุขจังหวัดปทุมธานี,สาขาพื้นที่(สระบุรี),ปทุมธานี,เมืองปทุมธานี,บางปรอก,สำนักงานสาธารณสุขจังหวัด
3,00004,สำนักงานสาธารณสุขจังหวัดพระนครศรีอยุธยา,สาขาพื้นที่(สระบุรี),พระนครศรีอยุธยา,พระนครศรีอยุธยา,หอรัตนไชย,สำนักงานสาธารณสุขจังหวัด
4,00005,สำนักงานสาธารณสุขจังหวัดอ่างทอง,สาขาพื้นที่(สระบุรี),อ่างทอง,เมืองอ่างทอง,บ้านอิฐ,สำนักงานสาธารณสุขจังหวัด


In [47]:
hospital.isnull().sum()     # ดูค่าว่าง

hcode          0
name           1
zone_name      4
province       0
amphur         2
tambon       209
name5          3
dtype: int64

###**name**

In [48]:
hospital[hospital['name'].isnull()]

,hcode,name,zone_name,province,amphur,tambon,name5
15453,26276,NaN,สาขาพื้นที่(สุราษฎร์ธานี),ชุมพร,เมืองชุมพร,NaN,คลินิกเอกชน


In [49]:
hospital['name'] = hospital['name'].fillna('ไม่ระบุ')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [50]:
hospital.isnull().sum()     # ดูค่าว่าง

hcode          0
name           0
zone_name      4
province       0
amphur         2
tambon       209
name5          3
dtype: int64

###**zone_name**

In [51]:
set(hospital['zone_name'])

{'08',
 nan,
 'สาขาพื้นที่(ขอนแก่น)',
 'สาขาพื้นที่(นครราชสีมา)',
 'สาขาพื้นที่(นครสวรรค์)',
 'สาขาพื้นที่(พิษณุโลก)',
 'สาขาพื้นที่(ระยอง)',
 'สาขาพื้นที่(ราชบุรี)',
 'สาขาพื้นที่(สงขลา)',
 'สาขาพื้นที่(สระบุรี)',
 'สาขาพื้นที่(สุราษฎร์ธานี)',
 'สาขาพื้นที่(อุดรธานี)',
 'สาขาพื้นที่(อุบลราชธานี)',
 'สาขาพื้นที่(เชียงใหม่)',
 'สาขาพื้นที่กรุงเทพมหานคร'}

In [52]:
hospital[(hospital['zone_name'].isnull()) | (hospital['zone_name']=='08')]   

,hcode,name,zone_name,province,amphur,tambon,name5
16800,40864,โรงพยาบาลสรรพสิทธิ์อินเตอร์,NaN,อุบลราชธานี,เมืองอุบลราชธานี,ในเมือง,โรงพยาบาลเอกชน
16801,40886,สำนักงานป้องกันควบคุมโรคที่ 8 จังหวัดอุดรธานี\...,08,อุดรธานี,เมืองอุดรธานี,NaN,ศูนย์บริการสาธารณสุข
16803,41092,โรงพยาบาลส่งเสริมสุขภาพตำบลหนองจานพัฒนา,NaN,นครราชสีมา,หนองบุญมาก,หนองบุนนาก,โรงพยาบาลส่งเสริมสุขภาพตำบล
16805,77609,หน่วยบริการปฐมภูมิหัวทุ่ง,NaN,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,หน่วยบริการปฐมภูมิ
16810,77651,โรงพยาบาลโคกโพธิ์ไชย,NaN,ขอนแก่น,เมืองขอนแก่น,บ้านโคก,โรงพยาบาลทั่วไป


In [53]:
hospital['zone_name'][16800] = 'สาขาพื้นที่(อุบลราชธานี)'     # แทนค่าว่าง ด้วยค่าที่มีความเป็นไปได้
hospital['zone_name'][16801] = 'สาขาพื้นที่(อุดรธานี)'
hospital['zone_name'][16803] = 'สาขาพื้นที่(นครราชสีมา)'
hospital['zone_name'][16805] = 'สาขาพื้นที่(ขอนแก่น)'
hospital['zone_name'][16810] = 'สาขาพื้นที่(ขอนแก่น)'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [54]:
set(hospital['zone_name'])

{'สาขาพื้นที่(ขอนแก่น)',
 'สาขาพื้นที่(นครราชสีมา)',
 'สาขาพื้นที่(นครสวรรค์)',
 'สาขาพื้นที่(พิษณุโลก)',
 'สาขาพื้นที่(ระยอง)',
 'สาขาพื้นที่(ราชบุรี)',
 'สาขาพื้นที่(สงขลา)',
 'สาขาพื้นที่(สระบุรี)',
 'สาขาพื้นที่(สุราษฎร์ธานี)',
 'สาขาพื้นที่(อุดรธานี)',
 'สาขาพื้นที่(อุบลราชธานี)',
 'สาขาพื้นที่(เชียงใหม่)',
 'สาขาพื้นที่กรุงเทพมหานคร'}

In [55]:
hospital.isnull().sum()

hcode          0
name           0
zone_name      0
province       0
amphur         2
tambon       209
name5          3
dtype: int64

###**amphur**

In [56]:
hospital[hospital['amphur'].isnull()]

,hcode,name,zone_name,province,amphur,tambon,name5
15064,25733,สารีคลินิกการแพทย์,สาขาพื้นที่กรุงเทพมหานคร,กรุงเทพมหานคร,NaN,NaN,คลินิกเอกชน
15180,25852,คลินิกแพทย์สุเชต,สาขาพื้นที่(สุราษฎร์ธานี),ภูเก็ต,NaN,NaN,คลินิกเอกชน


In [57]:
hospital['amphur'][15064] = 'ไม่ระบุ'     # แทนค่าว่าง ด้วยค่าที่มีความเป็นไปได้
hospital['amphur'][15180] = 'ถลาง'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [58]:
hospital.isnull().sum()

hcode          0
name           0
zone_name      0
province       0
amphur         0
tambon       209
name5          3
dtype: int64

###**tambon**

In [59]:
hospital['tambon'] = hospital['tambon'].fillna('ไม่ระบุ')     # แทนค่าว่างของ ตำบล ด้วย ไม่ระบุ

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [60]:
hospital.isnull().sum()

hcode        0
name         0
zone_name    0
province     0
amphur       0
tambon       0
name5        3
dtype: int64

###**name5 (type)**

In [61]:
hospital = hospital.rename(columns={'name5':'type'})  #     เปลี่ยนชื่อ คอลัมน์ name5 เป็น type
print(hospital.shape)
hospital.head()

(17103, 7)


,hcode,name,zone_name,province,amphur,tambon,type
0,00001,สำนักงานสาธารณสุขจังหวัดสมุทรปราการ,สาขาพื้นที่(ระยอง),สมุทรปราการ,เมืองสมุทรปราการ,ปากน้ำ,สำนักงานสาธารณสุขจังหวัด
1,00002,สำนักงานสาธารณสุขจังหวัดนนทบุรี,สาขาพื้นที่(สระบุรี),นนทบุรี,เมืองนนทบุรี,บางกระสอ,สำนักงานสาธารณสุขจังหวัด
2,00003,สำนักงานสาธารณสุขจังหวัดปทุมธานี,สาขาพื้นที่(สระบุรี),ปทุมธานี,เมืองปทุมธานี,บางปรอก,สำนักงานสาธารณสุขจังหวัด
3,00004,สำนักงานสาธารณสุขจังหวัดพระนครศรีอยุธยา,สาขาพื้นที่(สระบุรี),พระนครศรีอยุธยา,พระนครศรีอยุธยา,หอรัตนไชย,สำนักงานสาธารณสุขจังหวัด
4,00005,สำนักงานสาธารณสุขจังหวัดอ่างทอง,สาขาพื้นที่(สระบุรี),อ่างทอง,เมืองอ่างทอง,บ้านอิฐ,สำนักงานสาธารณสุขจังหวัด


In [62]:
set(hospital['type'])

{nan,
 'คลินิกเอกชน',
 'ศูนย์บริการสาธารณสุข',
 'ศูนย์วิชาการ',
 'ศูนย์สุขภาพชุมชน',
 'ศูนย์สุขภาพชุมชน ของ รพ.',
 'สถานบริการสาธารณสุขชุมชน',
 'สถานีอนามัย',
 'สำนักงานสาธารณสุขจังหวัด',
 'สำนักงานสาธารณสุขอำเภอ/กิ่งอำเภอ',
 'หน่วยบริการปฐมภูมิ',
 'โรงพยาบาล นอก สธ.',
 'โรงพยาบาล นอก สป.สธ.',
 'โรงพยาบาล/ศูนย์บริการสาธารณสุข สาขา',
 'โรงพยาบาลชุมชน',
 'โรงพยาบาลทั่วไป',
 'โรงพยาบาลศูนย์',
 'โรงพยาบาลส่งเสริมสุขภาพตำบล',
 'โรงพยาบาลส่งเสริมสุขภาพตำบล ',
 'โรงพยาบาลเอกชน'}

In [63]:
hospital[hospital['type'].isnull()]

,hcode,name,zone_name,province,amphur,tambon,type
16474,28268,คลินิกนายแพทย์สันติ,สาขาพื้นที่(สระบุรี),ลพบุรี,เมืองลพบุรี,ทะเลชุบศร,NaN
16484,28278,คลินิกหมอปาน,สาขาพื้นที่(สระบุรี),ลพบุรี,เมืองลพบุรี,กกโก,NaN
16486,28280,คลินิกหมอวิภาวดี,สาขาพื้นที่(สระบุรี),ลพบุรี,เมืองลพบุรี,กกโก,NaN


In [64]:
hospital['type'] = hospital['type'].fillna('คลินิกเอกชน')     # แทนค่าว่างของ type ด้วยค่าที่มีความเป็นไปได้

In [65]:
hospital['type'].replace({'โรงพยาบาลส่งเสริมสุขภาพตำบล ':'โรงพยาบาลส่งเสริมสุขภาพตำบล'},inplace=True)   #เปลี่ยนข้อมูลให้ตรงกันน

In [66]:
set(hospital['type'])

{'คลินิกเอกชน',
 'ศูนย์บริการสาธารณสุข',
 'ศูนย์วิชาการ',
 'ศูนย์สุขภาพชุมชน',
 'ศูนย์สุขภาพชุมชน ของ รพ.',
 'สถานบริการสาธารณสุขชุมชน',
 'สถานีอนามัย',
 'สำนักงานสาธารณสุขจังหวัด',
 'สำนักงานสาธารณสุขอำเภอ/กิ่งอำเภอ',
 'หน่วยบริการปฐมภูมิ',
 'โรงพยาบาล นอก สธ.',
 'โรงพยาบาล นอก สป.สธ.',
 'โรงพยาบาล/ศูนย์บริการสาธารณสุข สาขา',
 'โรงพยาบาลชุมชน',
 'โรงพยาบาลทั่วไป',
 'โรงพยาบาลศูนย์',
 'โรงพยาบาลส่งเสริมสุขภาพตำบล',
 'โรงพยาบาลเอกชน'}

In [67]:
hospital.isnull().sum()

hcode        0
name         0
zone_name    0
province     0
amphur       0
tambon       0
type         0
dtype: int64

###**ตาราง hospital ที่ clean แล้ว**

In [68]:
print(hospital.shape)       
hospital.head()

(17103, 7)


,hcode,name,zone_name,province,amphur,tambon,type
0,00001,สำนักงานสาธารณสุขจังหวัดสมุทรปราการ,สาขาพื้นที่(ระยอง),สมุทรปราการ,เมืองสมุทรปราการ,ปากน้ำ,สำนักงานสาธารณสุขจังหวัด
1,00002,สำนักงานสาธารณสุขจังหวัดนนทบุรี,สาขาพื้นที่(สระบุรี),นนทบุรี,เมืองนนทบุรี,บางกระสอ,สำนักงานสาธารณสุขจังหวัด
2,00003,สำนักงานสาธารณสุขจังหวัดปทุมธานี,สาขาพื้นที่(สระบุรี),ปทุมธานี,เมืองปทุมธานี,บางปรอก,สำนักงานสาธารณสุขจังหวัด
3,00004,สำนักงานสาธารณสุขจังหวัดพระนครศรีอยุธยา,สาขาพื้นที่(สระบุรี),พระนครศรีอยุธยา,พระนครศรีอยุธยา,หอรัตนไชย,สำนักงานสาธารณสุขจังหวัด
4,00005,สำนักงานสาธารณสุขจังหวัดอ่างทอง,สาขาพื้นที่(สระบุรี),อ่างทอง,เมืองอ่างทอง,บ้านอิฐ,สำนักงานสาธารณสุขจังหวัด


###**รวมตาราง alldb เข้ากับตาราง hospital**

In [69]:
data_all = alldb.merge(hospital,how='left',left_on='site',right_on='hcode')     # เชื่อมตาราง
print(data_all.shape)
data_all.head()

(38172, 18)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,sex,age,hcode,name,zone_name,province,amphur,tambon,type
0,06223,kfpwuaqldj.png,2022-03-23,0,Xiaomi 21081111RG,13777,16:31:45,ปกติ (เหลืองใส),00000,ไม่ระบุ,54,13777,โรงพยาบาลศรีนครินทร์ มหาวิทยาลัยขอนแก่น,สาขาพื้นที่(ขอนแก่น),ขอนแก่น,เมืองขอนแก่น,ในเมือง,โรงพยาบาล นอก สธ.
1,12015,00859DB9-1BD9-4825-B246-3DC25EFD1924,2022-03-30,4,"iPhone12,3",06307,14:46:08,ปกติ (เหลืองใส),00001,หญิง,47,06307,โรงพยาบาลส่งเสริมสุขภาพตำบลบ้านเกาะ ตำบลบ้านดารา,สาขาพื้นที่(พิษณุโลก),อุตรดิตถ์,พิชัย,บ้านดารา,โรงพยาบาลส่งเสริมสุขภาพตำบล
2,91811,1376ACA2-AFA5-4457-967A-D2FCF908E6A5,2022-03-30,4,"iPhone14,3",05328,15:27:45,ปกติ (เหลืองใส),00001,หญิง,41,05328,โรงพยาบาลส่งเสริมสุขภาพตำบลบ้านโคกประสิทธิ์,สาขาพื้นที่(ขอนแก่น),กาฬสินธุ์,ฆ้องชัย,โคกสะอาด,โรงพยาบาลส่งเสริมสุขภาพตำบล
3,95821,1914EE67-CD8F-47EE-A968-2772911AA5DF,2022-03-30,3,"iPhone13,4",06302,10:16:46,ปกติ (เหลืองใส),00001,หญิง,55,06302,โรงพยาบาลส่งเสริมสุขภาพตำบลสองห้อง,สาขาพื้นที่(พิษณุโลก),อุตรดิตถ์,ฟากท่า,สองห้อง,โรงพยาบาลส่งเสริมสุขภาพตำบล
4,13361,1A219EE7-3647-4D7D-A8C0-AE638E69D889,2022-03-29,3,"iPhone9,4",05059,12:26:38,ปกติ (เหลืองใส),00001,ชาย,66,05059,โรงพยาบาลส่งเสริมสุขภาพตำบลบ้านดอนดู่,สาขาพื้นที่(ขอนแก่น),มหาสารคาม,นาดูน,หัวดง,โรงพยาบาลส่งเสริมสุขภาพตำบล


In [70]:
data_all.isnull().sum()

user             0
RowKey           0
date             0
ov               0
phone            0
site             0
time             0
urine            0
PartitionKey     0
sex              0
age              0
hcode           76
name            76
zone_name       76
province        76
amphur          76
tambon          76
type            76
dtype: int64

In [71]:
data_all[data_all['hcode'].isnull()]        # ดูข้อมูลที่เป็นค่าว่าง

,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,sex,age,hcode,name,zone_name,province,amphur,tambon,type
30,05039,6F6ED841-7574-4C1C-AF10-CC6E2658E718,2022-03-28,0,"iPhone11,8",45972,10:36:41,ขุ่น,00001,ไม่ระบุ,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN
790,05353,7A4BD7DC-5BD3-4928-B57F-CE7C86E32DAE,2022-03-19,4,"iPad13,1",75757,17:22:08,ขุ่น,00006,ไม่ระบุ,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN
915,05353,218CB548-E5F1-4F5D-87C5-36EC2256B7CB,2022-03-19,4,"iPad13,1",75757,17:20:05,ปกติ (เหลืองใส),00007,ไม่ระบุ,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1675,05353,7CDA4D3C-6E12-4CF8-A94C-76E5CE4E9202,2022-03-19,4,"iPad13,1",75757,17:14:16,ปกติ (เหลืองใส),00012,ไม่ระบุ,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1800,05353,6FC7AE57-7365-4DCC-A656-772CBDDDA882,2022-03-19,0,"iPad13,1",75757,17:15:16,ขุ่น,00013,ไม่ระบุ,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38167,99999,F45E78D3-7D89-47A3-8207-C705377E0E0A,2022-03-11,1,"iPhone14,2",99999,11:36:34,ขุ่น,99999,ไม่ระบุ,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38168,99999,lnlohgqckf.png,2022-03-10,0,Samsung SM-G998B,99999,18:10:29,มีตะกอน,99999,ไม่ระบุ,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38169,99999,vkdocpmjiv.png,2022-03-10,1,Samsung SM-G998B,99999,12:14:41,"ขุ่น, มีเม็ดเลือดแดง",99999,ไม่ระบุ,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38170,99999,vrvyfnvayq.png,2022-03-10,1,Samsung SM-G998B,99999,13:58:01,ปกติ (เหลืองใส),99999,ไม่ระบุ,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
data_all = data_all.dropna()    # ลบข้อมูลที่เป็นค่าว่างทิ้ง
print(data_all.shape)
data_all.head()

(38096, 18)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,sex,age,hcode,name,zone_name,province,amphur,tambon,type
0,06223,kfpwuaqldj.png,2022-03-23,0,Xiaomi 21081111RG,13777,16:31:45,ปกติ (เหลืองใส),00000,ไม่ระบุ,54,13777,โรงพยาบาลศรีนครินทร์ มหาวิทยาลัยขอนแก่น,สาขาพื้นที่(ขอนแก่น),ขอนแก่น,เมืองขอนแก่น,ในเมือง,โรงพยาบาล นอก สธ.
1,12015,00859DB9-1BD9-4825-B246-3DC25EFD1924,2022-03-30,4,"iPhone12,3",06307,14:46:08,ปกติ (เหลืองใส),00001,หญิง,47,06307,โรงพยาบาลส่งเสริมสุขภาพตำบลบ้านเกาะ ตำบลบ้านดารา,สาขาพื้นที่(พิษณุโลก),อุตรดิตถ์,พิชัย,บ้านดารา,โรงพยาบาลส่งเสริมสุขภาพตำบล
2,91811,1376ACA2-AFA5-4457-967A-D2FCF908E6A5,2022-03-30,4,"iPhone14,3",05328,15:27:45,ปกติ (เหลืองใส),00001,หญิง,41,05328,โรงพยาบาลส่งเสริมสุขภาพตำบลบ้านโคกประสิทธิ์,สาขาพื้นที่(ขอนแก่น),กาฬสินธุ์,ฆ้องชัย,โคกสะอาด,โรงพยาบาลส่งเสริมสุขภาพตำบล
3,95821,1914EE67-CD8F-47EE-A968-2772911AA5DF,2022-03-30,3,"iPhone13,4",06302,10:16:46,ปกติ (เหลืองใส),00001,หญิง,55,06302,โรงพยาบาลส่งเสริมสุขภาพตำบลสองห้อง,สาขาพื้นที่(พิษณุโลก),อุตรดิตถ์,ฟากท่า,สองห้อง,โรงพยาบาลส่งเสริมสุขภาพตำบล
4,13361,1A219EE7-3647-4D7D-A8C0-AE638E69D889,2022-03-29,3,"iPhone9,4",05059,12:26:38,ปกติ (เหลืองใส),00001,ชาย,66,05059,โรงพยาบาลส่งเสริมสุขภาพตำบลบ้านดอนดู่,สาขาพื้นที่(ขอนแก่น),มหาสารคาม,นาดูน,หัวดง,โรงพยาบาลส่งเสริมสุขภาพตำบล


In [73]:
data_all.isnull().sum()     # ดูค่าว่าง

user            0
RowKey          0
date            0
ov              0
phone           0
site            0
time            0
urine           0
PartitionKey    0
sex             0
age             0
hcode           0
name            0
zone_name       0
province        0
amphur          0
tambon          0
type            0
dtype: int64

## **ตารางข้อมูลพร้อมใช้งาน**

In [74]:
print(data_all.shape)   
data_all.head()

(38096, 18)


,user,RowKey,date,ov,phone,site,time,urine,PartitionKey,sex,age,hcode,name,zone_name,province,amphur,tambon,type
0,06223,kfpwuaqldj.png,2022-03-23,0,Xiaomi 21081111RG,13777,16:31:45,ปกติ (เหลืองใส),00000,ไม่ระบุ,54,13777,โรงพยาบาลศรีนครินทร์ มหาวิทยาลัยขอนแก่น,สาขาพื้นที่(ขอนแก่น),ขอนแก่น,เมืองขอนแก่น,ในเมือง,โรงพยาบาล นอก สธ.
1,12015,00859DB9-1BD9-4825-B246-3DC25EFD1924,2022-03-30,4,"iPhone12,3",06307,14:46:08,ปกติ (เหลืองใส),00001,หญิง,47,06307,โรงพยาบาลส่งเสริมสุขภาพตำบลบ้านเกาะ ตำบลบ้านดารา,สาขาพื้นที่(พิษณุโลก),อุตรดิตถ์,พิชัย,บ้านดารา,โรงพยาบาลส่งเสริมสุขภาพตำบล
2,91811,1376ACA2-AFA5-4457-967A-D2FCF908E6A5,2022-03-30,4,"iPhone14,3",05328,15:27:45,ปกติ (เหลืองใส),00001,หญิง,41,05328,โรงพยาบาลส่งเสริมสุขภาพตำบลบ้านโคกประสิทธิ์,สาขาพื้นที่(ขอนแก่น),กาฬสินธุ์,ฆ้องชัย,โคกสะอาด,โรงพยาบาลส่งเสริมสุขภาพตำบล
3,95821,1914EE67-CD8F-47EE-A968-2772911AA5DF,2022-03-30,3,"iPhone13,4",06302,10:16:46,ปกติ (เหลืองใส),00001,หญิง,55,06302,โรงพยาบาลส่งเสริมสุขภาพตำบลสองห้อง,สาขาพื้นที่(พิษณุโลก),อุตรดิตถ์,ฟากท่า,สองห้อง,โรงพยาบาลส่งเสริมสุขภาพตำบล
4,13361,1A219EE7-3647-4D7D-A8C0-AE638E69D889,2022-03-29,3,"iPhone9,4",05059,12:26:38,ปกติ (เหลืองใส),00001,ชาย,66,05059,โรงพยาบาลส่งเสริมสุขภาพตำบลบ้านดอนดู่,สาขาพื้นที่(ขอนแก่น),มหาสารคาม,นาดูน,หัวดง,โรงพยาบาลส่งเสริมสุขภาพตำบล
